# Building blocks of Clojure

## Metadata

In [97]:
; Using the with-meta function to add metadata to a map
(def untrusted (with-meta 
    {:command "delete-table" :subject "users"}
    {:safe false :io true})) ; Metadata

untrusted

{:command "delete-table", :subject "users"}

In [98]:
; Using the '^' macro to add metadata to a map

(def untrusted 
    ^{:safe false :io true} 
    {:command "delete-table" :subject "users"})

untrusted

{:command "delete-table", :subject "users"}

In [99]:
; If using the '^' macro with a list, 
; the metadata won't take effect at runtime

(def untrusted-not-working 
    ^{:safe false :io true} 
    (hash-map :command "delete-table" :subject "users"))


#'user/untrusted-not-working

In [100]:
; Metadata doesn't affect value equality

(def trusted {:command "delete-table" :subject "users"}) 
(= trusted untrusted) 

true

In [101]:
; Using the meta function to examine metadata
(meta untrusted)


{:safe false, :io true}

In [102]:
; When new values are created from those that have metadata, 
; the metadata is copied over to the new data

(def still-untrusted (assoc untrusted :complete? false))
(meta still-untrusted)


{:safe false, :io true}

In [103]:
; Functions and macros can also be defined with metadata

(defn ^{:safe true 
        :console true
        :doc "testing metadata for functions"} 
    testing-meta
    [] 
    (println "Hello from meta!"))


#'user/testing-meta

In [104]:
; Trying to use the meta function directly to check that 
; the metadata was set correctly won't work

(meta testing-meta)

nil

In [105]:
; To access the metadata,  pass the testing-meta var to the meta function.
(meta (var testing-meta))

{:ns #namespace[user], :name testing-meta, :file "NO_SOURCE_PATH", :column 1, :line 3, :console true, :safe true, :arglists ([]), :doc "testing metadata for functions"}

Defining a function without Java type hints

In [106]:
; Setting a warning to alert when "Reflection" is used
(set! *warn-on-reflection* true) 

(defn string-length [x] (.length x))


Reflection warning, NO_SOURCE_PATH:4:25 - reference to field length can't be resolved.


#'user/string-length

In [107]:
; Function call without type hints (slow)

(time (reduce + (map string-length (repeat 10000 "12345"))))

"Elapsed time: 54.126 msecs"


50000

Defining a function with Java type hints

In [108]:
(defn fast-string-length [^String x] (.length x)) 


#'user/fast-string-length

In [109]:
; Function call with type hints (fast)

(time (reduce + (map fast-string-length (repeat 10000 "12345"))))

"Elapsed time: 3.1097 msecs"


50000

In [110]:
; The type hints of args is stored as metadata

(meta (first (first (:arglists (meta #'fast-string-length))))) 


{:tag String}

Type hinting primitive types

In [111]:
; Getting the Java Class name of BigDecimal

(defn array-type [klass]
    (.getName (class (make-array klass 0)))) 

(array-type BigDecimal)

"[Ljava.math.BigDecimal;"

In [112]:
; Using the name "[Ljava.math.BigDecimal;" as type hint

(def bigdec-arr 
    ^"[Ljava.math.BigDecimal;" 
    (into-array BigDecimal [1.0M]))


#'user/bigdec-arr

## Java exceptions: try and throw


In [113]:
; A function that calculates the average of a collection of numbers

(defn average [numbers]
    (let [total (apply + numbers)]
        (/ total (count numbers))))

#'user/average

In [114]:
; Calling the average function with an empty sequence throws an exception

(average [])

Execution error (ArithmeticException) at user/average (REPL:5).
Divide by zero


class java.lang.ArithmeticException: 

In [115]:
; Wrapping the last function with a try/catch expression

(defn safe-average [numbers]
    (let [total (apply + numbers)]
        (try
            (/ total (count numbers))
        (catch ArithmeticException e 
            (println "Divided by zero!")
        0))))

(safe-average [])

Divided by zero!


0

In [116]:
; The general form of using try/catch/finally is straightforward:

"""
(try expr* catch-clause* finally-clause?)
"""

""

In [117]:
; Using several catch and a finally clauses

(try
    (print "Attempting division... ")
    (/ 1 0)
(catch RuntimeException e "Runtime exception!") 
(catch ArithmeticException e "DIVIDE BY ZERO!")  
(catch Throwable e "Unknown exception encountered!") 
(finally 
    (println "done.")))


Attempting division... done.


"Runtime exception!"

In [118]:
; The finally clause can be used without catch clauses
(try
    (print "Attempting division... ")
    (/ 1 0)
(finally
    (println "done.")))


Attempting division... done.


Execution error (ArithmeticException) at user/eval4359 (REPL:4).
Divide by zero


class java.lang.ArithmeticException: 

In [119]:
; Throwing exceptions
(throw (Exception. "this is an error!"))

Execution error at user/eval4361 (REPL:2).
this is an error!


class java.lang.Exception: 

## Functions

### Function definitions

In [120]:
(use '[clojure.repl :only (doc)])

nil

In [121]:
; Syntax of the defn macro
"""
(defn function-name
    doc-string?
    metadata-map?
    [parameter-list*]
    conditions-map?
    body-expressions*) 
"""

""

In [122]:
; A basic function definition

(defn total-cost [item-cost number-of-items]
    (* item-cost number-of-items))

#'user/total-cost

In [123]:
; The defn macro expands the last function definition as follows

(def total-cost (fn [item-cost number-of-items]
    (* item-cost number-of-items)))


#'user/total-cost

In [124]:
; Adding docstring at function definition
(defn total-cost 
    "return line-item total of the item and quantity provided" 
    [item-cost number-of-items]
    (* item-cost number-of-items))

(doc total-cost)

-------------------------
user/total-cost
([item-cost number-of-items])
  return line-item total of the item and quantity provided


nil

Adding metadata to functions

In [125]:
; Adding metadata without using ^
(meta (defn myfn-attr-map {:a 1} [])) 

{:arglists ([]), :a 1, :line 2, :column 7, :file "NO_SOURCE_PATH", :name myfn-attr-map, :ns #namespace[user]}

In [126]:
; Adding metadata with ^
(meta (defn ^{:a 1} myfn-metadata []))

{:a 1, :arglists ([]), :line 2, :column 7, :file "NO_SOURCE_PATH", :name myfn-metadata, :ns #namespace[user]}

In [127]:
; When adding duplicate metadata, the last version is used
(meta (defn ^{:a 1} myfn-both {:a 2 :b 3} [])) 

{:a 2, :arglists ([]), :b 3, :line 2, :column 7, :file "NO_SOURCE_PATH", :name myfn-both, :ns #namespace[user]}

In [128]:
; When adding duplicate docstrings, the last version is used
(meta (defn ^{:a 1 :doc "doc 1"} myfn-redundant-docs "doc 2" {:a 2 :b 3 :doc "doc 3"} [])) 

{:ns #namespace[user], :name myfn-redundant-docs, :file "NO_SOURCE_PATH", :column 7, :line 2, :b 3, :arglists ([]), :doc "doc 3", :a 2}

Adding conditional to functions

In [129]:
; Adding pre and post conditionals to functions
(defn item-total [price quantity discount-percentage]
    {:pre [(> price 0) (> quantity 0)]
     :post [(> % 0)]}
    (->> (/ discount-percentage 100)
         (- 1)
         (* price quantity)
         float))


#'user/item-total

In [130]:
; Testing pre conditionals with a valid input
(item-total 100 2 10)-

#function[clojure.core/-]

In [131]:
; Testing pre conditionals with an invalid input
(item-total 100 -2 10)

Execution error (AssertionError) at user/item-total (REPL:2).
Assert failed: (> quantity 0)


class java.lang.AssertionError: 

In [ ]:
; Testing the post conditional with an invalid input
(item-total 100 2 110)


Execution error (AssertionError) at user/item-total (REPL:2).
Assert failed: (> % 0)


class java.lang.AssertionError: 

Multiple 'arity' of functions

In [132]:
; The general form of overloaded functions is as follows
"""
(defn function-name 
    ;; Note that each argument+body pair is enclosed in a list.
    ([arg1] body-executed-for-one-argument-call)
    ([arg1 arg2] body-executed-for-two-argument-call)
    ;; More cases may follow.)
"""

""

In [133]:
; Example of an overloaded function
(defn total-cost 
    ([item-cost number-of-items]
        (* item-cost number-of-items))
    ([item-cost]
        (total-cost item-cost 1))) ; Calling the 2-ary version from the 1-ary one


#'user/total-cost

Variadic functions

In [134]:
; The general form of variadic functions is as follows
"""
(defn name-of-variadic-function [param-1 param-2 & rest-args]
    (body-of-function))
"""

""

In [135]:
; Using the & symbol to define a variadic function

(defn total-all-numbers [& numbers]
    (apply + numbers))

#'user/total-all-numbers

In [136]:
; A variadic function with other nonvariadic arities
(defn many-arities
    ([] 0)
    ([a] 1)
    ([a b c] 3)
    ([a b c & more] "variadic"))


#'user/many-arities

In [137]:
; >Testing it with 0 args
(many-arities)

0

In [138]:
; Testing it with 1 arg
(many-arities "one argument")

1

In [139]:
; Testing it with 2 args (this case is not defined)
(many-arities "two" "arguments")

Execution error (ArityException) at user/eval4398 (REPL:2).
Wrong number of args (2) passed to: user/many-arities


class clojure.lang.ArityException: 

In [140]:
; Testing it with 3 args
(many-arities "three" "argu-" "ments")

3

In [141]:
; Testing it with more than 3 args
(many-arities "many" "more" "argu-" "ments")

"variadic"

### Recursive functions

Self-Recursive functions

In [142]:
; A recursive function that will blow the stack with big args
(defn count-down [n]
    (when-not (zero? n)
        (when (zero? (rem n 100)) 
            (println "count-down:" n))
        (count-down (dec n)))) 

;(count-down 100000) ; This will throw StackOverflowError
(count-down 500)

count-down: 500
count-down: 400
count-down: 300
count-down: 200
count-down: 100


nil

In [143]:
; Rewriting the last function with recur, which won't blow the stack
(defn count-downr [n]
    (when-not (zero? n)
        (if (zero? (rem n 100))
            (println "count-down:" n))
        (recur (dec n))))

;(count-downr 100000) ; This won't throw StackOverflowError, but will kill the jupyter kernel. Try it at the REPL
(count-downr 500)


count-down: 500
count-down: 400
count-down: 300
count-down: 200
count-down: 100


nil

Mutually recursive functions

In [144]:
; Mutually recursive functions that can blow the stack
(declare hat)
(defn cat [n]
    (when-not (zero? n)
        (when (zero? (rem n 100))
            (println "cat:" n))
        (hat (dec n)))) ; Calling hat

(defn hat [n]
    (when-not (zero? n)
        (if (zero? (rem n 100))
            (println "hat:" n))
        (cat (dec n)))) ; Calling cat

; (cat 5000000) ; This will throw StackOverflowError
(cat 500)

cat: 500
cat: 400
cat: 300
cat: 200
cat: 100


nil

In [145]:
; Calling mutually recursive functions with the trampoline function

(declare hatt)
(defn catt [n]
    (when-not (zero? n)
        (when (zero? (rem n 100))
            (println "catt:" n))
        (fn [] (hatt (dec n)))))

(defn hatt [n]
    (when-not (zero? n)
        (when (zero? (rem n 100))
            (println "hatt:" n))
        (fn [] (catt (dec n)))))

; (trampoline catt 5000000) ; This won't throw StackOverflowError, but will kill the jupyter kernel. Try it at the REPL
(trampoline catt 500)


catt: 500
catt: 400
catt: 300
catt: 200
catt: 100


nil

In [146]:
; Internal workings of the trampoline function
(defn trampoline
    ([f]
        (let [ret (f)]
            (if (fn? ret) ; fn? means 'Is my argument a function?'
                (recur ret)
            ret)))
    ([f & args]
    (trampoline (fn [] (apply f args)))))

#'user/trampoline

Calling functions

In [147]:
; Standard function call
(+ 1 2 3 4 5)

15

In [148]:
; Applying a function to a sequence of args with apply

(def list-of-expenses [39.95M 39.95M 39.95M 39.95M 39.95M]) 
(apply + list-of-expenses) 


199.75M

### Higher order functions

every?

In [149]:
;Checking if all sequence elements evaluate to True with every?

(def bools [true true true false false])
(every? true? bools) 

false

some

In [150]:
;Checking if at least 1 sequence element evaluates to True with some

(def bools [true true true false false])
(some true? bools) 


true

constantly

In [151]:
; Returning a fixed constant with constantly

(def two (constantly 2)) ; same as (defn two [& more] 2)

#'user/two

In [152]:
(two 1)

2

In [153]:
(two :a :b :c)


2

complement

In [154]:
; A function that ckecks is the 1st arg is greater than teh 2nd
(defn greater? [x y]
    (> x y))

(greater? 10 5)

true

In [155]:
; Taking the original function and returning the logically opposite value with complement
(def smaller? (complement greater?))

(smaller? 5 10)

true

comp

In [156]:
; Composing functions with comp

(def opp-zero-str (comp str not zero?)) ; Checks if the argument is nonzero

(opp-zero-str 1)

"true"

partial

In [157]:
; A function that checks if some number is above a threshold
(defn above-threshold? [threshold number]
    (> number threshold))


#'user/above-threshold?

In [158]:
; Partially applying the above-threshold? function with threshold 5, 
; Then using it to filter a sequence

(filter (partial above-threshold? 5) [ 1 2 3 4 5 6 7 8 9])

(6 7 8 9)

memoize

In [159]:
; Calling an artificially slow function (sleeps 1s before computing the result)
(defn slow-calc [n m]
    (Thread/sleep 1000)
    (* n m))

(time (slow-calc 5 7))


"Elapsed time: 1000.9461 msecs"


35

In [160]:
; Creating a new function that uses the memoize function to cache past computations

(def fast-calc (memoize slow-calc))

#'user/fast-calc

In [161]:
; Calling it the 1st time (slow)

(time (fast-calc 5 7))

"Elapsed time: 1000.5854 msecs"


35

In [162]:
; Calling it the 2nd time (fast)

(time (fast-calc 5 7))

"Elapsed time: 0.18 msecs"


35

Writing higher order functions

In [163]:
; A mapping that will be sorted in various ways

(def users 
    [{:username "kyle"
      :firstname "Kyle"
      :lastname "Smith"
      :balance 175.00M ; Use BigDecimals for money!
      :member-since "2009-04-16"}
     {:username "zak"
      :firstname "Zackary"
      :lastname "Jones"
      :balance 12.95M
      :member-since "2009-02-01"}
     {:username "rob"
      :firstname "Robert"
      :lastname "Jones"
      :balance 98.50M
      :member-since "2009-03-30"}])


#'user/users

In [164]:
; Sort all the sequence by the user key

(sort-by :username users)

({:username "kyle", :firstname "Kyle", :lastname "Smith", :balance 175.00M, :member-since "2009-04-16"} {:username "rob", :firstname "Robert", :lastname "Jones", :balance 98.50M, :member-since "2009-03-30"} {:username "zak", :firstname "Zackary", :lastname "Jones", :balance 12.95M, :member-since "2009-02-01"})

In [165]:
;A HOF that sorts a sequence by an arbitrary key

(defn sorter-using [ordering-fn]
    (fn [collection]
        (sort-by ordering-fn collection)))


#'user/sorter-using

In [166]:
; A function that selects the lastname and firstname as keys

(defn lastname-firstname [user]
    [(user :lastname) (user :firstname)])

#'user/lastname-firstname

In [167]:
; A function that selects the balance as key

(defn balance [user] 
    (user :balance))

#'user/balance

In [168]:
; A function that selects the username as key

(defn username [user] 
    (user :username))

#'user/username

In [169]:
; Using the sorter HOF and the balance key function
; to get the users sorted by balance

(def poorest-first 
    (sorter-using balance))

(poorest-first users)

({:username "zak", :firstname "Zackary", :lastname "Jones", :balance 12.95M, :member-since "2009-02-01"} {:username "rob", :firstname "Robert", :lastname "Jones", :balance 98.50M, :member-since "2009-03-30"} {:username "kyle", :firstname "Kyle", :lastname "Smith", :balance 175.00M, :member-since "2009-04-16"})

In [170]:
; Using the sorter HOF and the username key function
; to get the users sorted by name

(def alphabetically 
    (sorter-using username))

(alphabetically users)

({:username "kyle", :firstname "Kyle", :lastname "Smith", :balance 175.00M, :member-since "2009-04-16"} {:username "rob", :firstname "Robert", :lastname "Jones", :balance 98.50M, :member-since "2009-03-30"} {:username "zak", :firstname "Zackary", :lastname "Jones", :balance 12.95M, :member-since "2009-02-01"})

In [171]:
; Using the sorter HOF and the lastname-firstname key function
; to get the users sorted by last name

(def last-then-firstname 
    (sorter-using lastname-firstname))

(last-then-firstname users)

({:username "rob", :firstname "Robert", :lastname "Jones", :balance 98.50M, :member-since "2009-03-30"} {:username "zak", :firstname "Zackary", :lastname "Jones", :balance 12.95M, :member-since "2009-02-01"} {:username "kyle", :firstname "Kyle", :lastname "Smith", :balance 175.00M, :member-since "2009-04-16"})

In [172]:
; The last function sorts like the following functions

(sort (map lastname-firstname users))

(["Jones" "Robert"] ["Jones" "Zackary"] ["Smith" "Kyle"])

### Anonymous functions

In [173]:
; Creating an anonymous function with the fn macro

(fn [item-cost number-of-items]
    (* item-cost number-of-items))

#function[user/eval4476/fn--4477]

In [174]:
; Using map with an anonymous function to get the join date of users

(map (fn [user] (user :member-since)) users)

("2009-04-16" "2009-02-01" "2009-03-30")

In [175]:
; Using the # reader macro as a shortcut for anonymous functions

(map #(% :member-since) users) ; The anonymous function is #(% :member-since)


("2009-04-16" "2009-02-01" "2009-03-30")

Applying anonymous functions to sequences of variable length


In [176]:
(#(vector %&) 1 2 3 4 5)

[(1 2 3 4 5)]

In [177]:
(#(vector %1 %&) 1 2 3 4 5)

[1 (2 3 4 5)]

In [178]:
(#(vector %1 %2 %&) 1 2 3 4 5)

[1 2 (3 4 5)]

In [179]:
(#(vector %1 %2 %&) 1 2)

[1 2 nil]

### Keywords

In [180]:
; Using keywords in maps

(def person {:username "zak"
             :balance 12.95
             :member-since "2009-02-01"})


#'user/person

In [181]:
; Finding the value associated to the username key

(person :username)


"zak"

In [182]:
; Finding the value associated to the username key - alternative syntax

(:username person)


"zak"

In [183]:
; An anonymous function used in the last section

(map #(% :member-since) users)


("2009-04-16" "2009-02-01" "2009-03-30")

In [184]:
; Rewriting the last function using the keyword as function

(map :member-since users)

("2009-04-16" "2009-02-01" "2009-03-30")

In [185]:
; Looking up a non-existing key

(:login person)

nil

In [186]:
; Setting up a default value in case of a non-existing key

(:login person :not-found)


:not-found

### Symbols

In [187]:
; Defining a map with symbol literals

(def expense {'name "Snow Leopard" 'cost 29.95M})


#'user/expense

In [188]:
; Using a symbol to look up a value of a map
(expense 'name)


"Snow Leopard"

In [189]:
; Using a symbol to look up a value of a map - alternative syntax
('name expense)

"Snow Leopard"

In [190]:
; All operations that can be done with keywords are also possible with symbols
('vendor expense :absent)

:absent

Vectors as functions of their indices

In [191]:
; A simple vector

(def names ["kyle" "zak" "rob"])


#'user/names

In [192]:
; Looking up a vector item by its index

(names 1)

"zak"

In [193]:
; Looking up a vector item by an invalid index

(names 10)


Execution error (IndexOutOfBoundsException) at user/eval4548 (REPL:3).
null


class java.lang.IndexOutOfBoundsException: 